### Registro de Voz

Registrar nuevas voces en el sistema de reconocimiento de hablantes.

**Pasos:** Importar dependencias → Configurar rutas → Cargar modelo → Grabar audio → Generar embedding → Guardar en BD

#### 1. Importar dependencias
Librerías para grabación, procesamiento de audio y generación de embeddings.

In [1]:
import os
import time
import torch
import torchaudio
import numpy as np
import sounddevice as sd
import soundfile as sf

from speechbrain.pretrained import EncoderClassifier


d:\work_jhonatan_becerra\speach-recognition\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
d:\work_jhonatan_becerra\speach-recognition\venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
d:\work_jhonatan_becerra\speach-recognition\venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has b

#### 2. Configurar directorios
Define rutas del modelo, base de datos de voces y directorio temporal.

In [2]:
BASE_DIR = os.path.abspath("..")

MODEL_DIR = os.path.join(
    BASE_DIR,
    "models",
    "spkrec-ecapa-voxceleb"
)

VOICES_DB = os.path.join(BASE_DIR, "voices_db")
AUDIO_TMP = os.path.join(BASE_DIR, "audio_tmp")

os.makedirs(VOICES_DB, exist_ok=True)
os.makedirs(AUDIO_TMP, exist_ok=True)

print("MODEL_DIR:", MODEL_DIR)
print("VOICES_DB:", VOICES_DB)


MODEL_DIR: d:\work_jhonatan_becerra\speach-recognition\models\spkrec-ecapa-voxceleb
VOICES_DB: d:\work_jhonatan_becerra\speach-recognition\voices_db


#### 3. Cargar modelo ECAPA-TDNN
Carga el modelo SpeechBrain pre-entrenado en VoxCeleb (genera embeddings de 192 dimensiones).

In [3]:
classifier = EncoderClassifier.from_hparams(
    source=MODEL_DIR,
    savedir=MODEL_DIR,
    run_opts={"device": "cpu"}
)

print("Modelo cargado correctamente")


Modelo cargado correctamente


#### 4. Definir parámetros de audio
16 kHz, mono, 10 segundos de duración.

In [4]:
SAMPLE_RATE = 16_000
CHANNELS = 1
DURATION_SEC = 10

AUDIO_PATH = os.path.join(AUDIO_TMP, "registro.wav")


#### 5. Grabar voz desde micrófono
Captura 10 segundos de audio y guarda como WAV.

In [10]:
print("Grabando voz por 10 segundos...")
audio = sd.rec(
    int(DURATION_SEC * SAMPLE_RATE),
    samplerate=SAMPLE_RATE,
    channels=CHANNELS,
    dtype="float32"
)
sd.wait()

sf.write(AUDIO_PATH, audio, SAMPLE_RATE)
print("Audio capturado en:", AUDIO_PATH)


Grabando voz por 10 segundos...
Audio capturado en: d:\work_jhonatan_becerra\speach-recognition\audio_tmp\registro.wav


#### 6. Cargar y normalizar audio
Convierte a mono y normaliza amplitud.

In [11]:
waveform, sr = torchaudio.load(AUDIO_PATH)

assert sr == SAMPLE_RATE, "Sample rate incorrecto"

waveform = waveform.mean(dim=0, keepdim=True)
waveform = waveform / torch.max(torch.abs(waveform))

print("Waveform listo:", waveform.shape)


Waveform listo: torch.Size([1, 160000])


#### 7. Generar embedding de voz
Extrae vector de 192 dimensiones que representa las características únicas de la voz.

In [12]:
with torch.no_grad():
    embedding = classifier.encode_batch(waveform)

embedding = embedding.squeeze().cpu().numpy()

print("Embedding generado:", embedding.shape)


Embedding generado: (192,)


#### 8. Guardar embedding en base de datos
Solicita ID de la persona y guarda el embedding como `{persona_id}.npy` en voices_db.

In [13]:
persona_id = input("Ingrese el identificador de la persona (ej: jhonatan): ").strip()

assert persona_id, "ID inválido"

save_path = os.path.join(VOICES_DB, f"{persona_id}.npy")
np.save(save_path, embedding)

print("Voz registrada correctamente en:", save_path)


Voz registrada correctamente en: d:\work_jhonatan_becerra\speach-recognition\voices_db\jhonatan.npy
